In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pickle


from src.multiclass_svm import multiclassSVM
from src.HOG import *
from src.generate_kernel import gaussianKernel

# Importing data

In [2]:
train = pd.read_csv('./data/Xtr.csv', sep = ",", header = None)
test = pd.read_csv('./data/Xte.csv', sep = ",", header = None)

trainOutput = pd.read_csv('./data/Ytr.csv', sep = ",")

train.drop(train.columns[len(train.columns)-1], axis=1, inplace=True)
test.drop(test.columns[len(test.columns)-1], axis=1, inplace=True)
combined = train.append(test)
combined.reset_index(inplace=True)
combined.drop('index',inplace=True,axis=1)

# Create Histogram Of Gradients

In [3]:
combined_HOG = createHOG(storeImages(combined))

# Generate Gaussian kernel of the HOG

In [4]:
sigma = 60
K = gaussianKernel(combined_HOG, sigma)

# Make predictions with multiclass SVM

In [5]:
newtrain = combined_HOG[:5000]
newtest = combined_HOG[5000:]
newtrainOutput = trainOutput['Prediction']

In [6]:
mSVM = multiclassSVM(K = K)

In [7]:
mSVM.fit(newtrain, newtrainOutput)

     pcost       dcost       gap    pres   dres
 0:  4.6644e+06 -4.9408e+07  9e+07  4e-01  3e-13
 1:  3.0471e+06 -5.5826e+06  9e+06  2e-16  1e-12
 2:  5.3848e+05 -8.2987e+05  1e+06  2e-16  2e-13
 3:  7.7073e+04 -1.0179e+05  2e+05  2e-16  7e-14
 4:  9.9264e+03 -1.5119e+04  3e+04  2e-16  3e-14
 5:  8.7692e+02 -2.6474e+03  4e+03  2e-16  1e-14
 6: -1.8707e+02 -6.5019e+02  5e+02  2e-16  4e-15
 7: -2.7234e+02 -3.1111e+02  4e+01  2e-16  2e-15
 8: -2.7715e+02 -2.8077e+02  4e+00  2e-16  9e-16
 9: -2.7793e+02 -2.7822e+02  3e-01  2e-16  9e-16
10: -2.7802e+02 -2.7804e+02  1e-02  2e-16  9e-16
11: -2.7803e+02 -2.7803e+02  6e-04  2e-16  9e-16
12: -2.7803e+02 -2.7803e+02  2e-05  2e-16  9e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  4.4128e+06 -3.4754e+07  5e+07  1e-01  3e-13
 1:  1.9491e+06 -3.5108e+06  6e+06  1e-02  2e-13
 2:  3.3608e+05 -5.9845e+05  9e+05  2e-16  1e-13
 3:  4.8804e+04 -6.9001e+04  1e+05  2e-16  6e-14
 4:  5.9229e+03 -1.0363e+04  2e+04  2e-16  2e-1

In [8]:
final_predictions, _, _ = mSVM.predict(newtest, newtest.index)

# Dump to CSV

In [9]:
IdTest = np.array([i for i in range(1, 1 + len(test))])
output = [int(x) for x in final_predictions]
df_output = pd.DataFrame()
df_output['Id'] = IdTest
df_output['Prediction'] = output
df_output[['Id','Prediction']].to_csv('./Yte.csv', sep = ",", index=False)